# Introduction/Business Problem

### Opportunity to identify opened businesses in Lima Perú for entrepeneurs that would like to open up a new business in Peru's capital.

### Target Audience: Stakeholders who would like to invest in a new business in Peru.

### Data:  Foursquare information to pull Lima available businesses to filter and cluster by category.  All businesses around Lima, PE -12.0621065 -77.0365256

In [260]:
address = 'Lima, PE'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-12.0621065 -77.0365256


In [261]:
search_query = ''
radius = 500
print(search_query + ' .... OK!')

 .... OK!


In [262]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WW5OCLYRGMNVBSRPSIATIYB1UP5ZYVFTDV1VLL4SZO35NGSB&client_secret=AGHMK4CJ2HNHUPDPNON2UGLQDX4KPT3QM3H3FF0A1Z4CULV0&ll=-12.0621065,-77.0365256&v=20180604&query=&radius=500&limit=500'

In [263]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c4fda84dd5797192b908c6d'},
 'response': {'confident': False,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'primary': True,
      'shortName': 'Park'}],
    'hasPerk': False,
    'id': '4bb6697f6edc76b008e6301c',
    'location': {'address': 'Av 28 de Julio',
     'cc': 'PE',
     'city': 'Lima',
     'country': 'Perú',
     'crossStreet': 'esq. Garcilaso de la Vega, Lima',
     'distance': 48,
     'formattedAddress': ['Av 28 de Julio (esq. Garcilaso de la Vega, Lima)',
      'Lima',
      '01',
      'Perú'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -12.062538359858769,
       'lng': -77.03651874783061}],
     'lat': -12.062538359858769,
     'lng': -77.03651874783061,
     'postalCode': '01',
     'state': 'Lima'},
    'name': 'Parque de la Ex

In [264]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'shortName': 'Park', 'primary': True, 'plura...",False,4bb6697f6edc76b008e6301c,Av 28 de Julio,PE,Lima,Perú,"esq. Garcilaso de la Vega, Lima",48,"[Av 28 de Julio (esq. Garcilaso de la Vega, Li...","[{'lng': -77.03651874783061, 'lat': -12.062538...",-12.062538,-77.036519,NaN,01,Lima,Parque de la Exposición,v-1548737156,NaN
1,[],False,4d2f53e16fc6f04d14388945,NaN,PE,Lima,Perú,NaN,15,"[Lima, Perú]","[{'lng': -77.03638942824972, 'lat': -12.062079...",-12.062080,-77.036389,NaN,NaN,Lima,Snack Govinda,v-1548737156,NaN
2,"[{'shortName': 'Theater', 'primary': True, 'pl...",False,5037d3dae4b03267a8243305,Av. Paseo de la República Cuadra 4 s/n Parque ...,PE,Cercado de Lima,Perú,Av. 28 de Julio,82,[Av. Paseo de la República Cuadra 4 s/n Parque...,"[{'lng': -77.03576493833978, 'lat': -12.062084...",-12.062085,-77.035765,NaN,1,Lima,Escuela Nacional Superior de Arte Dramático - ...,v-1548737156,NaN
3,"[{'shortName': 'Concert Hall', 'primary': True...",False,514e12c6e4b01766057ea459,NaN,PE,Lima,Perú,NaN,109,"[Lima, Perú]","[{'lng': -77.03554540789176, 'lat': -12.062308...",-12.062308,-77.035545,NaN,NaN,Lima,Anfiteatro del Parque de la Exposición,v-1548737156,NaN
4,"[{'shortName': 'Entertainment', 'primary': Tru...",False,505dbe91e4b067ab6c126cc2,Parque de La Exposición,PE,Cercado de Lima,Perú,Av. 28 de Julio,27,"[Parque de La Exposición (Av. 28 de Julio), Ce...","[{'lng': -77.03634444749349, 'lat': -12.061940...",-12.061941,-77.036344,NaN,01,Lima,Lima Vive Rock,v-1548737156,NaN


In [265]:
dataframe.shape

(115, 19)

In [266]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Parque de la Exposición,Park,Av 28 de Julio,PE,Lima,Perú,"esq. Garcilaso de la Vega, Lima",48,"[Av 28 de Julio (esq. Garcilaso de la Vega, Li...","[{'lng': -77.03651874783061, 'lat': -12.062538...",-12.062538,-77.036519,NaN,01,Lima,4bb6697f6edc76b008e6301c
1,Snack Govinda,None,NaN,PE,Lima,Perú,NaN,15,"[Lima, Perú]","[{'lng': -77.03638942824972, 'lat': -12.062079...",-12.062080,-77.036389,NaN,NaN,Lima,4d2f53e16fc6f04d14388945
2,Escuela Nacional Superior de Arte Dramático - ...,College Theater,Av. Paseo de la República Cuadra 4 s/n Parque ...,PE,Cercado de Lima,Perú,Av. 28 de Julio,82,[Av. Paseo de la República Cuadra 4 s/n Parque...,"[{'lng': -77.03576493833978, 'lat': -12.062084...",-12.062085,-77.035765,NaN,1,Lima,5037d3dae4b03267a8243305
3,Anfiteatro del Parque de la Exposición,Concert Hall,NaN,PE,Lima,Perú,NaN,109,"[Lima, Perú]","[{'lng': -77.03554540789176, 'lat': -12.062308...",-12.062308,-77.035545,NaN,NaN,Lima,514e12c6e4b01766057ea459
4,Lima Vive Rock,General Entertainment,Parque de La Exposición,PE,Cercado de Lima,Perú,Av. 28 de Julio,27,"[Parque de La Exposición (Av. 28 de Julio), Ce...","[{'lng': -77.03634444749349, 'lat': -12.061940...",-12.061941,-77.036344,NaN,01,Lima,505dbe91e4b067ab6c126cc2
5,Invita Peru,Food Truck,NaN,PE,NaN,Perú,NaN,102,[Perú],"[{'lng': -77.0366761801772, 'lat': -12.0630165...",-12.063017,-77.036676,NaN,NaN,NaN,51cc7f71498ecf4dd835982d
6,La Gloria,Spanish Restaurant,Restaurantes de Lujo - Mistura,PE,Lima,Perú,NaN,88,"[Restaurantes de Lujo - Mistura, Lima, Perú]","[{'lng': -77.03734, 'lat': -12.062126, 'label'...",-12.062126,-77.037340,NaN,NaN,Lima,4e173b491fc7d99db5c7c8a8
7,Pizza Hut,Pizza Place,C.C. Real Plaza Lima,PE,Cercado de Lima,Perú,NaN,168,"[C.C. Real Plaza Lima, Cercado de Lima, 1, Perú]","[{'lng': -77.035231590271, 'lat': -12.06127070...",-12.061271,-77.035232,NaN,1,Lima,4c1c2467eac020a1f61f46c2
8,Fiesta,South American Restaurant,Av. Reducto 1278 Miraflores,PE,Lima,Perú,NaN,88,"[Av. Reducto 1278 Miraflores, Lima, Perú]","[{'lng': -77.03734, 'lat': -12.062126, 'label'...",-12.062126,-77.037340,NaN,NaN,Lima,4e173aa01fc7d99db5c7c853
9,Cevichería El Paisa,Seafood Restaurant,NaN,PE,Lima,Perú,NaN,60,"[Lima, Perú]","[{'lng': -77.03646252194288, 'lat': -12.061565...",-12.061565,-77.036463,NaN,NaN,Lima,4ef20784d3e3e38057d1a91a


In [267]:
dataframe_filtered.name

0                                Parque de la Exposición
1                                          Snack Govinda
2      Escuela Nacional Superior de Arte Dramático - ...
3                 Anfiteatro del Parque de la Exposición
4                                         Lima Vive Rock
5                                            Invita Peru
6                                              La Gloria
7                                              Pizza Hut
8                                                 Fiesta
9                                    Cevichería El Paisa
10                          Museo de Arte de Lima - MALI
11                            Auditorio AFP Integra MALI
12                                            Real Plaza
13                                           Body secret
14                                     Torre Corporativa
15                           Museo Metropolitano de Lima
16                                    D'Caleta. Rest-Bar
17                             